In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, fitGMM

from auctions.webutils import *

from scipy.stats import loguniform
import copy

import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Magma256
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


from open_spiel.python.examples.ubc_cma import analyze_checkpoint

output_notebook()


Loading BokehJS ...

In [5]:
# load experiment
experiment = Experiment.objects.get(name='new_configs_nov_29')

In [ ]:
# For each run, I want to know: ApproxNashConv, Auction Length, Revenue, Potential Function, Game Name, Game Mod

In [21]:
records = []
for run in experiment.equilibriumsolverrun_set.all():
    record = dict(game_name=run.game.name, potential=run.config.get('potential_function', 'None'), seed=run.config.get('seed'), run_name=run.name)
    
    # Get the last eval
    final_checkpoint = run.equilibriumsolverruncheckpoint_set.last()
    record = {**record, **analyze_checkpoint(final_checkpoint)}
    records.append(record)

In [22]:
df = pd.DataFrame.from_records(records)

In [17]:
df.sort_values(['game_name', 'potential']).set_index([df.index, 'game_name', 'potential'], drop=True)

,,,seed,p0_utility,p0_payment,p1_utility,p1_payment,total_welfare,total_revenue,auction_lengths,common_allocations
,game_name,potential,,,,,,,,,
0,confused_national.json,None,102,4850.333676,1950.184968,5723.987021,1699.183028,14223.688693,3649.367996,5.785064,"[((2, 0, 1), 4261), ((0, 2, 1), 3041), ((1, 0,..."
1,confused_national.json,None,100,4418.853084,2779.870088,4796.892019,2165.147685,14160.762876,4945.017773,11.027803,"[((2, 0, 1), 3870), ((1, 2, 1), 1721), ((1, 0,..."
2,confused_national.json,None,101,5080.499087,1962.081848,5434.349250,1663.862880,14140.793066,3625.944728,5.648581,"[((2, 0, 1), 4225), ((0, 2, 1), 2741), ((1, 0,..."
3,confused_national.json,None,100,5039.063329,2202.229542,5314.222087,1705.862804,14261.377762,3908.092346,6.893611,"[((2, 0, 1), 3898), ((1, 0, 2), 1621), ((0, 2,..."
4,confused_national.json,None,101,4628.076672,2524.698283,4966.721880,2126.499064,14245.995899,4651.197347,9.549210,"[((2, 0, 1), 4349), ((1, 0, 2), 1607), ((1, 2,..."
5,confused_national.json,None,102,4541.294607,2514.148249,5167.426193,1855.748290,14078.617338,4369.896539,7.933607,"[((2, 0, 1), 4200), ((1, 0, 2), 1647), ((1, 2,..."
6,confused_national.json,auction_length,100,4329.730545,2482.334155,5513.613537,1852.883863,14178.562100,4335.218019,8.048400,"[((2, 0, 1), 4068), ((1, 0, 2), 1536), ((0, 2,..."
7,confused_national.json,auction_length,101,4642.854963,2676.992237,4906.542746,1939.376854,14165.766800,4616.369091,9.054900,"[((2, 0, 1), 3776), ((1, 0, 2), 1621), ((1, 2,..."
8,confused_national.json,auction_length,102,4666.406779,2503.913457,5163.559744,1911.362772,14245.242751,4415.276229,8.949610,"[((2, 0, 1), 4168), ((1, 0, 2), 1599), ((1, 2,..."


In [12]:
# TODO: Inner groupby on revenue, auction_length
df.groupby(['game_name', 'potential'])['auction_lengths'].describe()[['min', 'max']]

min        max
game_name                        potential                              
confused_national.json           None                5.648581  11.027803
                                 auction_length      8.048400   9.054900
                                 neg_auction_length  7.494099   9.011101
                                 neg_revenue         7.117629   8.310155
                                 revenue             8.805719  10.201020
regional_vs_national.json        None                1.001601   3.002502
                                 auction_length      1.001501   2.000100
                                 neg_auction_length  1.001601   2.983987
                                 neg_revenue         1.001601   2.879828
                                 revenue             1.001100   3.242549
regional_vs_national_signal.json None                1.269946   2.892014
                                 auction_length      1.001301   2.786815
                                 neg_auction_length  1.000000   2.588953
                                 neg_revenue         2.432059   3.091273
                                 revenue             1.792054   3.047424

In [ ]:
Game.objects.get(name='regional_vs_national.json').

In [36]:
def delta(grp):
    lengths = grp.sort_values('potential')['auction_lengths'].values
    return lengths[1] - lengths[0]

valid_potentials = ['None', 'auction_length']
df.query('potential in @valid_potentials and run_name.str.contains("ppo_3")').groupby(['game_name', 'seed']).apply(delta)

game_name                         seed
confused_national.json            100    -2.979403
                                  101    -0.494310
                                  102     1.016003
regional_vs_national.json         100    -0.002000
                                  101     0.001601
                                  102    -1.002402
regional_vs_national_signal.json  100    -1.208081
                                  101     1.033338
                                  102    -1.890713
dtype: float64